<a href="https://colab.research.google.com/github/aNo2mousNo2/pytorch_expamples/blob/master/bert/colab/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install transformers
!pip install pytorch_lightning
!pip install fugashi
!pip install ipadic

!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar -zxf ldcc-20140209.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-07-11 15:47:33--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.1’

ldcc-20140209.tar.g 100%[===================>]   8.44M  6.09MB/s    in 1.4s    

2022-07-11 15:47:35 (6.09 MB/s) - ‘ldcc-20140209.tar.gz.1’ saved [8855190/8855190]



In [10]:
import random
import glob
from tqdm import tqdm
import torch
import torchtext
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

In [11]:
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [12]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
bert_sc = bert_sc.cuda()

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [15]:
def encode(text_list, labels=None):
  encoding = tokenizer(text_list, padding = 'longest', return_tensors='pt')
  if labels is not None:
    encoding['labels'] = torch.tensor(labels)
    encoding = { k: v.cuda() for k, v in encoding.items() }
    return encoding

In [16]:
text_list = [
             'この映画は面白かった。',
             'この映画の最後にはがっかりさせられた。',
             'この映画を見て幸せな気持ちになった。'
]
label_list = [1, 0, 1]

# encoding = tokenizer(
#     text_list,
#     padding = 'longest',
#     return_tensors='pt'
# )
# encoding = { k: v.cuda() for k, v in encoding.items() }
# labels = torch.tensor(label_list).cuda()
encode(text_list)
with torch.no_grad():
  output = bert_sc.forward(**encoding)

scores = output.logits
labels_predicted = scores.argmax(-1)
num_correct = (labels_predicted == labels).sum().item()
accuracy = num_correct / labels.size(0)

print('# score size')
print(scores.size())
print('# predicted labels:')
print(labels_predicted)
print('# accuracy:')
print(accuracy)

# score size
torch.Size([3, 2])
# predicted labels:
tensor([0, 0, 0], device='cuda:0')
# accuracy:
0.3333333333333333


In [20]:
encoding = encode(text_list, label_list)

output = bert_sc(**encoding)
loss = output.loss
print(loss)

tensor(0.7264, device='cuda:0', grad_fn=<NllLossBackward0>)


In [23]:
category_list = [
                 'dokujo-tsushin',
                 'it-life-hack',
                 'kaden-channel',
                 'livedoor-homme',
                 'movie-enter',
                 'peachy',
                 'smax',
                 'sports-watch',
                 'topic-news'
]

tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

max_length = 128
dataset_for_loader = []
for label, category in enumerate(tqdm(category_list)):
  for file in glob.glob(f'./text/{category}/{category}*'):
    lines = open(file).read().splitlines()
    text = '\n'.join(lines[3:])
    encoding = tokenizer(text, max_length=max_length, padding='max_length', truncation=True)
    encoding['labels'] = label
    encoding = {k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)

100%|██████████| 9/9 [00:40<00:00,  4.51s/it]


In [24]:
print(dataset_for_loader[0])

{'input_ids': tensor([    2,    36,  5233,   106,  6905,    14,  9727, 28504,    58,    16,
          322,    16,    26,  3871,    38,  1018, 20652,  5972,     5,     9,
            6,   569,   335,     5,  3834, 28601,  2375,    23,  1832,   573,
           35,   811,  1151,   258,  3834, 28601,  2375,    14,  6905,    12,
         6669,    11, 20064,    16,    21,    10,     5,     9,   134,    19,
          174,     8, 13198,  8107,    12,   901,     5,  6675,    49,  2198,
            5, 13169, 31363,     6, 12277,   251,  5797,    64,  7173,    18,
         3676,     7,     9,  5498,    80,   124,    18,   108, 28833,    11,
        25064, 28491, 28517,    16,    21,    10,     8,   373,   366,     6,
         9898, 29296, 28535,     5,  4697,   225,  3834, 28601,  2375,     5,
         6669,     9, 18659,   118,  3267,     8,   893,   604,  2183,     9,
        10892,     7, 14450, 28477,     6,  9898, 29296, 28535,     9,    73,
         2712,  2122,  1080,   308,    10,   140, 

In [25]:
random.shuffle(dataset_for_loader)
n = len(dataset_for_loader)
n_train = int(0.6 * n)
n_val = int(0.2 * n)
dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train:n_train + n_val]
dataset_test = dataset_for_loader[n_train + n_val:]

dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)


In [29]:
class BertForSequenceClassification_pl(pl.LightningModule):
  def __init__(self, model_name, num_labels, lr):
    super().__init__()
    self.save_hyperparameters()
    self.bert_sc = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
  
  def training_step(self, batch, batch_idx):
    output = self.bert_sc(**batch)
    loss = output.loss
    self.log('train_loss', loss)
    return loss
  
  def validation_step(self, batch, batch_idx):
    output = self.bert_sc(**batch)
    val_loss = output.loss
    self.log('val_loss', val_loss)
  
  def test_step(self, batch, batch_idx):
    labels = batch.pop('labels')
    output = self.bert_sc(**batch)
    labels_predicted = output.logits.argmax(-1)
    num_correct = (labels_predicted == labels).sum().item()
    accuracy = num_correct / labels.size(0)
    self.log('accuracy', accuracy)
  
  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [30]:
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

trainer = pl.Trainer(
    gpus=1,
    max_epochs=10,
    callbacks = [checkpoint]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
model = BertForSequenceClassification_pl(MODEL_NAME, num_labels=9, lr=1e-5)

trainer.fit(model, dataloader_train, dataloader_val)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]